# Part 3 - Evaluate simulations with experimental analogue

### First load some references

In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using System.IO;
using System.Data;
using System.Globalization;
using System.Threading;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Platform.LinAlg;
using BoSSS.Application.XNSE_Solver.PhysicalBasedTestcases.PrintingNip;
Init();

### Init Database etc.

In [ ]:
string ProjectName = "SFB1194_K65_Part3";

In [ ]:
BoSSSshell.WorkflowMgm.Init(ProjectName);

In [ ]:
static var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

### Load Simulations

In [ ]:
var sessions = myDb.Sessions.Where(s => s.ProjectName == ProjectName).ToArray();

In [ ]:
sessions.Count()

### Setup Post-Processing

In [ ]:
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Clear();

In [ ]:
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("PressureRange", Postprocessing.PressureRange);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("VelocityXMax", Postprocessing.VelocityXMax);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("Massflux", Postprocessing.Massflux);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("NipShearRate", Postprocessing.NipShearRate);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("NipShearRateCylinder", Postprocessing.NipShearRateCylinder);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("ShearStressCylinder", Postprocessing.ShearStressCylinder);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("NipShearRateSubstrate", Postprocessing.NipShearRateSubstrate);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("ShearStressSubstrate", Postprocessing.ShearStressSubstrate);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("ViscousDissipation", Postprocessing.ViscousDissipation); 
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("PositionOfSynchronousFlow", Postprocessing.PositionOfSynchronousFlow);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("PositionOfStagnatingFlow", Postprocessing.PositionOfStagnatingFlow);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("dPdXatSynchronousPoint", Postprocessing.dPdXatSynchronousPoint);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("dPdXatStagnationPoint", Postprocessing.dPdXatStagnationPoint);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("dPdXatNip", Postprocessing.dPdXatNip);
BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Add("dPdXatConstantX", Postprocessing.dPdXatConstantX);

### Create Datatable

In [ ]:
DataTable tab;
if(!File.Exists("./PrintingNip/Files/PrintingNip-Part3Export.csv")){
    tab = sessions.GetSessionTable(BoSSSshell.WorkflowMgm.AdditionalSessionTableColums.Select(kv => new Tuple<string, Func<ISessionInfo, object>>(kv.Key, kv.Value)).ToArray());
    tab.ToCSVFile("./PrintingNip/Files/PrintingNip-Part3Export.csv", ColSep: '%');
} else{
    tab = TableExtensions.FromCSVFile("./PrintingNip/Files/PrintingNip-Part3Export.csv", ColSep: '%');
}

### Export Datatable

In [ ]:
string[] Columns = new string[] {"PressureRange", "VelocityXMax", "Massflux", "NipShearRate", "NipShearRateCylinder", "ShearStressCylinder", "NipShearRateSubstrate", "ShearStressSubstrate", "ViscousDissipation", "PositionOfSynchronousFlow",  "PositionOfStagnatingFlow", "dPdXatSynchronousPoint", "dPdXatStagnationPoint", "dPdXatNip", "dPdXatConstantX"};
string[] Labels = new string[] {"Pressure", "Velocity", "Massflux", "Shearrate (Nip)", "Shearrate (Cylinder)", "Shearforce (Cylinder)", "Shearrate (Substrate)", "Shearforce (Substrate)", "Dissipation", "$x_{sync}$", "$x_{stag}$", @"$\\\frac{\\\partial p}{\\\partial x}_{sync}$", @"$\\\frac{\\\partial p}{\\\partial x}_{stag}$", @"$\\\frac{\\\partial p}{\\\partial x}_{nip}$", @"$\\\frac{\\\partial p}{\\\partial x}$"};
string[] Units = new string[] {@"$\\\left[\\\frac{N}{m^2}\\right]$", @"$\\\left[\\\frac{m}{s}\\right]$", @"$\\\left[\\\frac{kg}{s}\\right]$", @"$\\\left[\\\frac{1}{s}\\right]$", @"$\\\left[\\\frac{1}{s}\\right]$", @"$\\\left[N\\right]$", @"$\\\left[\\\frac{1}{s}\\right]$", @"$\\\left[N\\right]$", @"$\\\left[W\\right]$", @"$\\\left[m\\right]$", @"$\\\left[m\\right]$", @"$\\\left[\\\frac{N}{m^3}\\right]$", @"$\\\left[\\\frac{N}{m^3}\\right]$", @"$\\\left[\\\frac{N}{m^3}\\right]$", @"$\\\left[\\\frac{N}{m^3}\\right]$"}; // some escape character nonsense

In [ ]:
var tab2export = tab.ExtractColumns(Columns.ToList().Prepend("id:V_Wall").Prepend("id:P_Diff").Prepend("id:Radius").Prepend("id:delta").Prepend("id:Raster").Prepend("SessionName").ToArray());
tab2export.ToHTMLFile("PrintingNip-Part3.html", Path.GetFullPath("./PrintingNip/Files"));
tab2export.ToCSVFile("./PrintingNip/Files/PrintingNip-Part3.csv", ColSep: ';');

### Plot Correlation - Constant Nip Width

Plot all Properties, we looked at before as functions of nip width. Remember, this time around the pressure b.c. is such that the stagnation point should remain constant

In [ ]:
var tab2plt = tab.ExtractRows((i,row) => Convert.ToDouble(row["id:P_Diff"]) > 0.0 && Convert.ToDouble(row["id:delta"]).ApproxEqual(1e-5)); // arbitrary nip width

In [ ]:
List<Plot2Ddata> plts = new List<Plot2Ddata>();
foreach(string col in Columns){
    int k = Columns.IndexOf(col);
    var plt = tab2plt.ToPlot("id:V_Wall", col, ColName_GroupSelection: new string[] { "id:Raster" });    
    plt.LogX   = true;
    plt.LogY   = true;
    plt.Xlabel = @"Printing Velocity Width $\\\left[\\\frac{m}{s}\\right]$";
    plt.Ylabel = Labels[k] + " " + Units[k];
    plt.LabelTitleFont = 32;
    plt.LabelFont = 24;
    plt.LegendFont = 18;
    plt.lmargin = 12;
    plt.bmargin = 4;
    plt.ShowLegend = true;    
    plt.LegendBox = true;
    if(plt.Regression().Average(kv => kv.Value) > 0){
        plt.LegendAlignment = new string[]{"i", "l", "t"};
        plt.LegendSwap = true;    
    } else{
        plt.LegendAlignment = new string[]{"i", "r", "t"};
        plt.LegendSwap = false;    
    }
    for(int n= 0; n < plt.dataGroups.Count(); n++){
        plt.dataGroups[n].Format.LineWidth = 2;
        plt.dataGroups[n].Format.PointSize = 1;
        plt.dataGroups[n].Name = "Raster $ = " + Convert.ToDouble(plt.dataGroups[n].Name.Split("id:Raster").Last()).ToString() + @" \\\frac{1}{cm}$";
    }
    plt.ModFormat();
    plts.Add(plt);
}

In [ ]:
using(StreamWriter stw = new StreamWriter("./PrintingNip/Output/SlopesOverRaster.txt")){
    foreach(var plt in plts){
        var slopes = plt.Regression();
        stw.WriteLine("<===================================================>");
        stw.WriteLine(plt.Ylabel + ":");
        slopes.ForEach(kvp => stw.WriteLine("\t" + kvp.Key + " : " + kvp.Value));
        stw.WriteLine("<===================================================>");
    }
}

In [ ]:
int k = 0;
foreach(var plt in plts){
    var gp = new Gnuplot();
    gp.PlotLogSlope(plt, format: new PlotFormat(lineColor: (LineColors)7));
    gp.Cmd("set key box width -9");
    plt.ToGnuplot(gp);
    gp.PlotCairolatex().SaveTo("./PrintingNip/Figures/Part3/H0_"+Columns[k]+".tex");
    k++;
}

### Plot Correlation - Constant Raster

In [ ]:
var tab2plt = tab.ExtractRows((i,row) => Convert.ToDouble(row["id:P_Diff"]) > 0.0 && Convert.ToDouble(row["id:Raster"]).ApproxEqual(80)); // arbitrary nip width

In [ ]:
List<Plot2Ddata> plts = new List<Plot2Ddata>();
foreach(string col in Columns){
    int k = Columns.IndexOf(col);
    var plt = tab2plt.ToPlot("id:V_Wall", col, ColName_GroupSelection: new string[] { "id:delta" });    
    plt.LogX   = true;
    plt.LogY   = true;
    plt.Xlabel = @"Printing Velocity $\\\left[\\\frac{m}{s}\\right]$";
    plt.Ylabel = Labels[k] + " " + Units[k];
    plt.LabelTitleFont = 32;
    plt.LabelFont = 24;
    plt.LegendFont = 18;
    plt.lmargin = 12;
    plt.bmargin = 4;
    plt.ShowLegend = true;    
    plt.LegendBox = true;
    if(plt.Regression().Average(kv => kv.Value) > 0){
        plt.LegendAlignment = new string[]{"i", "l", "t"};
        plt.LegendSwap = true;    
    } else{
        plt.LegendAlignment = new string[]{"i", "r", "t"};
        plt.LegendSwap = false;    
    }     
    for(int n= 0; n < plt.dataGroups.Count(); n++){
        plt.dataGroups[n].Format.LineWidth = 2;
        plt.dataGroups[n].Format.PointSize = 1;
        plt.dataGroups[n].Name = @"$\\\varepsilon = " + Convert.ToDouble(plt.dataGroups[n].Name.Split("id:delta").Last()).ToString("0e0").Replace("e","\\\\times 10^{") + "}" + @" m$";
    }
    plt.ModFormat();
    plts.Add(plt);
}

In [ ]:
using(StreamWriter stw = new StreamWriter("./PrintingNip/Output/SlopesOverNipWidth.txt")){
    foreach(var plt in plts){
        var slopes = plt.Regression();
        stw.WriteLine("<===================================================>");
        stw.WriteLine(plt.Ylabel + ":");
        slopes.ForEach(kvp => stw.WriteLine("\t" + kvp.Key + " : " + kvp.Value));
        stw.WriteLine("<===================================================>");
    }
}

In [ ]:
int k = 0;
foreach(var plt in plts){
    var gp = new Gnuplot();
    gp.PlotLogSlope(plt, format: new PlotFormat(lineColor: (LineColors)7));
    gp.Cmd("set key box width -5");
    plt.ToGnuplot(gp);
    gp.PlotCairolatex().SaveTo("./PrintingNip/Figures/Part3/R0_"+Columns[k]+".tex");
    k++;
}

### Test plotting

In [ ]:
// Test code - see TestPDF.tex for output preview
// int i = 1;
// var gp = new Gnuplot();
// gp.PlotLogSlope(plts.Pick(1), format: new PlotFormat(lineColor: (LineColors)7));
// gp.Cmd("set key box width -5");
// plts.Pick(i).ToGnuplot(gp);
// gp.PlotCairolatex().SaveTo("./Figures/Test/Part3.tex");